# Deploying image classification as REST API

This notebook shows how to publish a trained image classification model as a Rest API service. We will start with local deployment (which uses docker), and then illustrate how easy it is using the same approach to instead publish to an Azure Container Service (ACS) with Kubernetes container management. 


## Prerequisites
- All numbered scripts until `5_evaluate.py` have to be executed as is described in part 1 of the documentation. This trains the DNN/SVM model which will get deployed.
- We assume the reader is familiar with the excellent deployment section of the [IRIS tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/preview/tutorial-classifying-iris-part-3) and the [Model management setup how-to guide](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-configuration).
- Local deployment requires a Docker server to be installed and running on the local machine. See the [Docker homepage](https://www.docker.com) or the AML Workbench [Troubleshooting guide](https://docs.microsoft.com/en-us/azure/machine-learning/preview/known-issues-and-troubleshooting-guide) for installation instructions.
Note that, at the time of writing, Docker supports the Windows 10 Operating System but not Windows Server 2016 which is running on the Window's Deep Learning Virtual Machines. 


## Rest API Implementation
The Rest API is implemented in the `deploymain.py` script using these three functions:
- Init(): loads the trained DNN/SVM model into memory after the Rest API is deployed.
- Run(): takes an image (in base64 encoding) as input, runs the full image classification pipeline including evaluating the DNN/SVM model, and returns the classification scores as json encoded string.
- Main(): can be used locally to test and debug the init() and run() functions before deployment. It creates a random 5x5 pixel RGB image, converts it to a base64 encoded string, which is then used as input to the run() function.

During deployment, a [swagger specification](https://en.wikipedia.org/wiki/OpenAPI_Specification) file can optionally be specified which defines how to describe and consume the web service. This swagger file is called `deployserviceschema.json` and was automatically created running `deploymain.py` (see the call to `generate_schema()`).

## Initialization
Various files are required by the web-service, including, among others, the trained DNN or SVM models. These files are copied in the code below to a local folder called *tmp*. During deployment, this folder is re-created on the node which runs the Rest API: note how the script `deploymain.py` loads files from that *tmp* folder. 

In [ ]:
import sys, os
sys.path.append(".")
sys.path.append("..")
sys.path.append("libraries")
sys.path.append("../libraries")
from helpers import *
from PARAMETERS import procDir

# Set files source and destination information
model_files_to_copy   = ["cntk_fixed.model", "cntk_refined.model", "lutId2Label.pickle", "svm.np"]
code_files_to_copy    = ["deploymain.py", "deployserviceschema.json"]
library_files_to_copy = ["helpers.py", "helpers_cntk.py", "utilities_CVbasic_v2.py", "utilities_general_v2.py"]
model_folder   = procDir 
code_folder    = "../scripts"
library_folder = "../libraries"
deploy_folder  = os.path.join(code_folder, "tmp")

# Copy files
makeDirectory(deploy_folder)
copyFiles(model_files_to_copy,   model_folder,   deploy_folder)
copyFiles(library_files_to_copy, library_folder, deploy_folder)
copyFiles(code_files_to_copy,    code_folder,    deploy_folder)
print("Files copied to deployment folder: " + deploy_folder)

## Local deployment
We now describe how to deploy the trained model as a Rest API which runs inside a docker container on the local machine.
All commands shown below need to be executed from the AML Workbench command prompt which can be opened via **File**->**Open Command Prompt**. 

### Steps:
1. Change to the directory which contains the scripts:
   ```sh 
   cd scripts
   ```
2. Set deployment target to local. Note that the Workbench might prompt the user to login first using the command `az login`. 
   ```sh
   az ml env local
   ```  

3. Create Rest API service (this can take 10-20 minutes). Use the command below as-is if `classifier=svm` in `deploymain.py`, but if `classifier=dnn` then change *cntk_fixed.model* to *cntk_refined.model*.
   ```sh
   az ml service create realtime -c ../aml_config/conda_dependencies.yml -f deploymain.py -s deployserviceschema.json -n imgclassapi1 -v -r python -d tmp/helpers.py -d tmp/helpers_cntk.py -d tmp/utilities_CVbasic_v2.py -d tmp/utilities_general_v2.py -d tmp/svm.np -d tmp/lutId2Label.pickle --model-file tmp/cntk_fixed.model
   ```
4. Test the Rest API directly from the command prompt:
   ```sh
   az ml service run realtime -i imgclassapi1 -d "{\"input_df\": [{\"image base64 string\": \"iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAIAAAACDbGyAAAAFElEQVR4nGP8//8/AxJgYkAFpPIB6vYDBxf2tWQAAAAASUVORK5CYII=\"}]}"
   ```

5. Obtain information of the Rest API by running:
   ```sh
   az ml service usage realtime -i imgclassapi1
   ```
Note that this also outputs a *scoring url* which looks like (but is not identical) to *http://127.0.0.1:32773/score*. This url can be used in script `6_callWebservice.py` which shows how to call the REST API from python rather than the command line.

## Cloud deployment
Deploying the image classification service to Azure is very similar compared to the local deployment described above. The main difference is that an Azure Container Service needs to be created first. The actual deployment steps are then identical:

### Steps
- Simply follow the all the instructions in the [model management setup how-to guide](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-configuration) to set up the cloud deployment environment. Be careful when creating an Azure Container Service to delete it again once not needed anymore.
- Then run all steps except for step 2 as explained in the *local deployment* section. 

**Example:**
1. Register the environment provider:
   ```sh
   az provider register -n Microsoft.MachineLearningCompute
   az provider register -n Microsoft.ContainerRegistry 
   ```

2. Create an ACS cluster (may take 10-20 minutes to be completely provisioned):
   ```sh
   az ml env setup --cluster -l westcentralus -n acsdeployment
   ```
3. Find resource group and cluster name in the list of compute targets (look for entry with "current mode: cluster"):
   ```sh
   az ml env list to see compute targes
   ```
4. Specify which compute target to use for 'cloud' deployment (this also returns the url for the Kubernetes dashboard): 
   ```sh
    az ml env set -n pabuehledeployvienna -g pabuehledeployviennarg
   ```    
5. Switch from local to cluster deployment:
   ```sh
   az ml env cluster
   ```             
6. Repeat all steps in the *local deployment* section except for step 2 which sets the local deployment target. The Rest API name should resemble *imgclassapi1.pabuehledeployvienna-bf329684.westcentralus*.    
      
7. Update script `6_` with the new scoring url and with the service key obtained by running: 
   ```sh
   az ml service keys realtime -i imgclassapi1.pabuehledeployvienna-bf327884.westcentralus
   ```

## Clean-up
Finally, we need delete all files in the local (temporary) deployment folder *tmp*. If this is not done, then the project directory will be above the size-limit of 25 MBytes.

In [19]:
deleteFiles(model_files_to_copy,   deploy_folder)
deleteFiles(library_files_to_copy, deploy_folder)
deleteFiles(code_files_to_copy,    deploy_folder)
print("Files deleted from deployment folder : " + deploy_folder)

../scripts\tmp\cntk_fixed.model
../scripts\tmp\cntk_refined.model
../scripts\tmp\lutId2Label.pickle
../scripts\tmp\svm.np
../scripts\tmp\helpers.py
../scripts\tmp\helpers_cntk.py
../scripts\tmp\utilities_CVbasic_v2.py
../scripts\tmp\utilities_general_v2.py
../scripts\tmp\deploymain.py
../scripts\tmp\deployserviceschema.json
Files deleted from deployment folder : ../scripts\tmp


## Debugging
These commands or guidelines can be helpful for understanding e.g. what caused a deployment error or why the deployed Rest API is not working:
- Inspect the docker log for errors:
    ```bash
    docker ps -a  # this shows all docker containers with their respective ID
    docker logs <containerid>
    ```
- List all deployed services: 
    ```bash
    az ml service list realtime
    ```
- Wrap all code running in the Rest API within *try...except* statments. See the run() and init() functions in `deploymain.py` as an example. This way, should an error occured during an API call, a description of this error is returned to the user.
